In [ ]:
import requests
import json
import uuid

# Configuração
BASE_URL_ADK_SERVER = "https://adk-server-health-894271896157.europe-west1.run.app"
BASE_URL_FASTAPI = "https://fastapi-server-health-894271896157.europe-west1.run.app"


def get_all_apps(base_url=BASE_URL_ADK_SERVER):
    """
    Recupera a lista de todos os aplicativos disponíveis na API ADK.
    
    Returns:
        list - Lista de aplicativos.
    """
    response = requests.get(f"{base_url}/list-apps")
    return response.json()

all_apps = get_all_apps()
all_apps

['compliance_agent',
 'parallel_analyzer_agent',
 'search_agent',
 'sequential_analyzer_agent',
 'simple_prescription_agent']

In [2]:
# Load data 
with open("../data/patients_ready_to_dashboard.json", "r", encoding="utf-8") as f:
    df_ready_to_dashboard = json.load(f)

In [6]:
# Simple API wrapper functions
def analyze_simple(health_data: str):
    """Simple prescription analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/simple",
        json={"health_data": health_data}
    )
    return response.json()

def analyze_parallel(health_data: str):
    """Parallel multi-agent analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/parallel", 
        json={"health_data": health_data}
    )
    return response.json()

def analyze_sequential(health_data: str):
    """Sequential health analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/sequential",
        json={"health_data": health_data}
    )
    return response.json()

def analyze_all(health_data: str):
    """All three analyses at once"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/all",
        json={"health_data": health_data}
    )
    return response.json()

print("API wrapper functions loaded:")
print("- analyze_simple(health_data)")
print("- analyze_parallel(health_data)")
print("- analyze_sequential(health_data)")
print("- analyze_all(health_data)")

API wrapper functions loaded:
- analyze_simple(health_data)
- analyze_parallel(health_data)
- analyze_sequential(health_data)
- analyze_all(health_data)


In [8]:
sample = df_ready_to_dashboard[:10]
inputs_to_llm = [patient['admission_str_input_to_llm'] for patient in sample]

# Test with first patient
if inputs_to_llm:
    patient_data = inputs_to_llm[2]
    
    # Test simple analysis
    result = analyze_simple(patient_data)
    print("Simple analysis result:")
    print(json.dumps(result, indent=2))
else:
    print("No patient data available for testing")

Simple analysis result:
{
  "status": "success",
  "data": {
    "state": {},
    "user_id": "api_user",
    "session_id": "s_c89c67ae",
    "results_criticality": {
      "level": "HIGH",
      "description": "Carbidopa-Levodopa is an oral medication. The current prescription specifies administration into the RIGHT EYE, which is an incorrect route. This prescription is potentially very dangerous."
    }
  },
  "message": "Simple prescription analysis completed successfully"
}


In [9]:
from tqdm import tqdm 

sample = df_ready_to_dashboard[:100]
inputs_to_llm = [patient['admission_str_input_to_llm'] for patient in sample]

outputs_parallel = []
for patient_data in tqdm(inputs_to_llm, desc="Analyzing patients"):
    result = analyze_parallel(patient_data)
    outputs_parallel.append(result)

Analyzing patients: 100%|██████████| 100/100 [04:51<00:00,  2.91s/it]


In [10]:
outputs_simple = []
for patient_data in tqdm(inputs_to_llm, desc="Analyzing patients"):
    result = analyze_simple(patient_data)
    outputs_simple.append(result)

Analyzing patients: 100%|██████████| 100/100 [02:47<00:00,  1.68s/it]


In [32]:
import pandas as pd

df_ready_to_dashboard_ = pd.DataFrame(df_ready_to_dashboard)

In [35]:
outputs_simple_data = [result['data']['results_criticality'] for result in outputs_simple]
outputs_simple_data_ = []
for i, element in enumerate(outputs_simple_data):
    x = {
        "subject_id.simple": i,
        "level": element['level'],
        "description_general": element['description']
    }
    outputs_simple_data_.append(x)

outputs_parallel_data = [result['data']['synthesized_results_criticality'] for result in outputs_parallel]
outputs_parallel_data_ = []
for i, element in enumerate(outputs_parallel_data):
    x = {
        "subject_id.parallel": i,
        "level_drug": element['level_drug'],
        "level_dose": element['level_dose'],
        "level_route": element['level_route'],
        "description_drug_dose_route": element['description']
    }
    outputs_parallel_data_.append(x)

In [36]:
df_outputs_simple_data = pd.DataFrame(outputs_simple_data_)
df_outputs_parallel_data = pd.DataFrame(outputs_parallel_data_)

In [37]:
df_full = pd.concat(
    [df_ready_to_dashboard_.reset_index(drop=True), 
     df_outputs_simple_data.reset_index(drop=True),
     df_outputs_parallel_data.reset_index(drop=True),
    ], axis=1
).drop(columns=["subject_id.simple", "subject_id.parallel"])

In [40]:
df_full

,subject_id,admission_id,admission_date,discharge_date,insurance,age,gender,language,marital_status,ethnicity,...,original_is_poisoned,original_poison_choice,admission_str,admission_str_input_to_llm,level,description_general,level_drug,level_dose,level_route,description_drug_dose_route
0,0,169938,4323585540000,4324203900000,Medicare,None,M,None,None,WHITE,...,False,None,Subject ID: 0\nAdmission ID: 169938\nAdmission...,Subject ID: 0\nAdmission ID: 169938\nAdmission...,MEDIUM,The prescription is for MethylPREDNISolone Sod...,HIGH,HIGH,HIGH,The synthesized assessment indicates a HIGH le...
1,1,180546,7301011740000,7301628240000,Medicare,None,M,SPAN,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,False,None,Subject ID: 1\nAdmission ID: 180546\nAdmission...,Subject ID: 1\nAdmission ID: 180546\nAdmission...,HIGH,Mycophenolate Mofetil is not available as an e...,HIGH,HIGH,HIGH,All three agents identified a critical safety ...
2,2,198330,4493893500000,4494761400000,Private,None,M,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 2\nAdmission ID: 198330\nAdmission...,Subject ID: 2\nAdmission ID: 198330\nAdmission...,HIGH,"Carbidopa-Levodopa is an oral medication, and ...",HIGH,HIGH,HIGH,"All three specialist agents (drug, dose, and r..."
3,3,139932,5609405400000,5612182200000,Medicare,None,F,ENGL,MARRIED,WHITE,...,False,None,Subject ID: 3\nAdmission ID: 139932\nAdmission...,Subject ID: 3\nAdmission ID: 139932\nAdmission...,LOW,Rituximab ophthalmic solution is not present i...,HIGH,HIGH,HIGH,All three specialist agents raised HIGH concer...
4,4,171628,4484040540000,4484635200000,Private,None,M,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 4\nAdmission ID: 171628\nAdmission...,Subject ID: 4\nAdmission ID: 171628\nAdmission...,HIGH,The current prescription is for Nafcillin 1334...,LOW,MEDIUM,HIGH,Nafcillin itself is considered a low risk. How...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,103379,6341512440000,6342343200000,Private,None,F,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 995\nAdmission ID: 103379\nAdmissi...,Subject ID: 995\nAdmission ID: 103379\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
996,996,178513,4645774500000,4646635800000,Medicare,None,F,ENGL,MARRIED,WHITE,...,False,None,Subject ID: 996\nAdmission ID: 178513\nAdmissi...,Subject ID: 996\nAdmission ID: 178513\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
997,997,180546,7301011740000,7301628240000,Medicare,None,M,SPAN,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,False,None,Subject ID: 997\nAdmission ID: 180546\nAdmissi...,Subject ID: 997\nAdmission ID: 180546\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
998,998,146893,5068417740000,5068971000000,Medicare,None,F,ENGL,SINGLE,WHITE,...,True,route,Subject ID: 998\nAdmission ID: 146893\nAdmissi...,Subject ID: 998\nAdmission ID: 146893\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# transform to json list objects
df_full_json = df_full.to_dict(orient='records')

# Save this json file in ../data
with open("../data/df_full_dashboard.json", "w", encoding="utf-8") as f:
    json.dump(df_full_json, f, ensure_ascii=False, indent=2)

In [ ]:
# Load data 
with open("../data/df_full_dashboard.json", "r", encoding="utf-8") as f:
    teste = json.load(f)

In [ ]:
pd.DataFrame(teste).sample(10)

,subject_id,admission_id,admission_date,discharge_date,insurance,age,gender,language,marital_status,ethnicity,...,original_is_poisoned,original_poison_choice,admission_str,admission_str_input_to_llm,level,description_general,level_drug,level_dose,level_route,description_drug_dose_route
0,0,169938,4323585540000,4324203900000,Medicare,None,M,None,None,WHITE,...,False,None,Subject ID: 0\nAdmission ID: 169938\nAdmission...,Subject ID: 0\nAdmission ID: 169938\nAdmission...,MEDIUM,The prescription is for MethylPREDNISolone Sod...,HIGH,HIGH,HIGH,The synthesized assessment indicates a HIGH le...
1,1,180546,7301011740000,7301628240000,Medicare,None,M,SPAN,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,False,None,Subject ID: 1\nAdmission ID: 180546\nAdmission...,Subject ID: 1\nAdmission ID: 180546\nAdmission...,HIGH,Mycophenolate Mofetil is not available as an e...,HIGH,HIGH,HIGH,All three agents identified a critical safety ...
2,2,198330,4493893500000,4494761400000,Private,None,M,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 2\nAdmission ID: 198330\nAdmission...,Subject ID: 2\nAdmission ID: 198330\nAdmission...,HIGH,"Carbidopa-Levodopa is an oral medication, and ...",HIGH,HIGH,HIGH,"All three specialist agents (drug, dose, and r..."
3,3,139932,5609405400000,5612182200000,Medicare,None,F,ENGL,MARRIED,WHITE,...,False,None,Subject ID: 3\nAdmission ID: 139932\nAdmission...,Subject ID: 3\nAdmission ID: 139932\nAdmission...,LOW,Rituximab ophthalmic solution is not present i...,HIGH,HIGH,HIGH,All three specialist agents raised HIGH concer...
4,4,171628,4484040540000,4484635200000,Private,None,M,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 4\nAdmission ID: 171628\nAdmission...,Subject ID: 4\nAdmission ID: 171628\nAdmission...,HIGH,The current prescription is for Nafcillin 1334...,LOW,MEDIUM,HIGH,Nafcillin itself is considered a low risk. How...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,103379,6341512440000,6342343200000,Private,None,F,ENGL,SINGLE,WHITE,...,False,None,Subject ID: 995\nAdmission ID: 103379\nAdmissi...,Subject ID: 995\nAdmission ID: 103379\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
996,996,178513,4645774500000,4646635800000,Medicare,None,F,ENGL,MARRIED,WHITE,...,False,None,Subject ID: 996\nAdmission ID: 178513\nAdmissi...,Subject ID: 996\nAdmission ID: 178513\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
997,997,180546,7301011740000,7301628240000,Medicare,None,M,SPAN,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,False,None,Subject ID: 997\nAdmission ID: 180546\nAdmissi...,Subject ID: 997\nAdmission ID: 180546\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN
998,998,146893,5068417740000,5068971000000,Medicare,None,F,ENGL,SINGLE,WHITE,...,True,route,Subject ID: 998\nAdmission ID: 146893\nAdmissi...,Subject ID: 998\nAdmission ID: 146893\nAdmissi...,NaN,NaN,NaN,NaN,NaN,NaN


: 